In [13]:
import numpy as np
import random
import json
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from helpers import bag_of_words, tokenize, stem
from model import NeuralNet
from dataset import ChatDataset

c:\Users\Mohammad\Anaconda3\envs\mybase\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
all_words = []
tags = []
xy = []
intent = None

with open('intents.json', 'r') as f:
    intents = json.load(f)

In [3]:
for intent in intents['intents']:
    tag = intent['intent']
    tags.append(tag)
    for pattern in intent['text']:
        w = tokenize(pattern)
        all_words.extend(w)
        xy.append((w, tag))

In [6]:
print(all_words, "\n\n")
print(tags, "\n\n")
print(xy, "\n\n")

['Hi', 'Hi', 'there', 'Hola', 'Hello', 'Hello', 'there', 'Hya', 'Hya', 'there', 'My', 'user', 'is', 'Adam', 'This', 'is', 'Adam', 'I', 'am', 'Adam', 'It', 'is', 'Adam', 'My', 'user', 'is', 'Bella', 'This', 'is', 'Bella', 'I', 'am', 'Bella', 'It', 'is', 'Bella', 'How', 'are', 'you', '?', 'Hi', 'how', 'are', 'you', '?', 'Hello', 'how', 'are', 'you', '?', 'Hola', 'how', 'are', 'you', '?', 'How', 'are', 'you', 'doing', '?', 'Hope', 'you', 'are', 'doing', 'well', '?', 'Hello', 'hope', 'you', 'are', 'doing', 'well', '?', 'Good', 'thanks', '!', 'My', 'user', 'is', 'Adam', 'Good', 'thanks', '!', 'This', 'is', 'Adam', 'Good', 'thanks', '!', 'I', 'am', 'Adam', 'Good', 'thanks', '!', 'It', 'is', 'Adam', 'Great', 'thanks', '!', 'My', 'user', 'is', 'Bella', 'Great', 'thanks', '!', 'This', 'is', 'Bella', 'Great', 'thanks', '!', 'I', 'am', 'Bella', 'Great', 'thanks', '!', 'It', 'is', 'Bella', 'What', 'is', 'my', 'name', '?', 'What', 'do', 'you', 'call', 'me', '?', 'Who', 'do', 'you', 'think', 'I', 'a

In [7]:
ignore_words = ['?', '.', '!']
all_words = [stem(w) for w in all_words if w not in ignore_words]
all_words = sorted(set(all_words))
tags = sorted(set(tags))

In [8]:
print(len(xy), "patterns")
print(len(tags), "tags:", tags)
print(len(all_words), "unique stemmed words:", all_words)

138 patterns
21 tags: ['Clever', 'CourtesyGoodBye', 'CourtesyGreeting', 'CourtesyGreetingResponse', 'CurrentHumanQuery', 'GoodBye', 'Greeting', 'GreetingResponse', 'NameQuery', 'NotTalking2U', 'PodBayDoor', 'PodBayDoorResponse', 'ProductQuery', 'RealNameQuery', 'SelfAware', 'Shutup', 'Swearing', 'Thanks', 'TimeQuery', 'UnderstandQuery', 'WhoAmI']
105 unique stemmed words: ["'s", ',', 'a', 'adam', 'adio', 'am', 'anyon', 'are', 'ask', 'awar', 'bay', 'be', 'bella', 'by', 'bye', 'call', 'camera', 'can', 'clever', 'commun', 'comprendo', 'consciou', 'cost', 'could', 'do', 'doe', 'door', 'enough', 'for', 'friend', 'fuck', 'geniou', 'get', 'girl', 'good', 'goodby', 'great', 'have', 'hello', 'help', 'hi', 'hola', 'hope', 'how', 'hya', 'i', 'identifi', 'in', 'intellig', 'is', 'it', 'know', 'later', 'me', 'mean', 'meant', 'much', 'my', "n't", 'name', 'not', 'of', 'off', 'ok', 'open', 'pleas', 'pod', 'price', 'prove', 'quiet', 'real', 'say', 'see', 'self', 'self-awar', 'shhh', 'shit', 'shut', 'spe

In [9]:
X_train = []
y_train = []
for (pattern_sentence, tag) in xy:
    bag = bag_of_words(pattern_sentence, all_words)
    X_train.append(bag)
    label = tags.index(tag)
    y_train.append(label)

In [10]:
X_train = np.array(X_train)
y_train = np.array(y_train)

In [11]:
num_epochs = 1000
batch_size = 8
learning_rate = 0.001
input_size = len(X_train[0])
hidden_size = 8
output_size = len(tags)
print(input_size, output_size)

105 21


In [14]:
dataset = ChatDataset(X_train, y_train)
train_loader = DataLoader(dataset=dataset,
                          batch_size=batch_size,
                          shuffle=True,
                          num_workers=0)

In [15]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [16]:
model = NeuralNet(input_size, hidden_size, output_size).to(device)

In [17]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [18]:
for epoch in range(num_epochs):
    for (words, labels) in train_loader:
        words = words.to(device)
        labels = labels.to(dtype=torch.long).to(device)
        
        # Forward pass
        outputs = model(words)
        # if y would be one-hot, we must apply
        # labels = torch.max(labels, 1)[1]
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    if (epoch+1) % 100 == 0:
        print (f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

Epoch [100/1000], Loss: 0.0274
Epoch [200/1000], Loss: 0.0051
Epoch [300/1000], Loss: 0.0004
Epoch [400/1000], Loss: 0.0001
Epoch [500/1000], Loss: 0.0005
Epoch [600/1000], Loss: 0.0001
Epoch [700/1000], Loss: 0.0001
Epoch [800/1000], Loss: 0.0000
Epoch [900/1000], Loss: 0.0000
Epoch [1000/1000], Loss: 0.0000


In [19]:
print(f'final loss: {loss.item():.4f}')

final loss: 0.0000


In [21]:
data = {
"model_state": model.state_dict(),
"input_size": input_size,
"hidden_size": hidden_size,
"output_size": output_size,
"all_words": all_words,
"tags": tags
}

FILE = "data.pth"
torch.save(data, FILE)

print(f'training complete. file saved to {FILE}')

training complete. file saved to data.pth
